In [ ]:
import glob

import numpy as np
from pylab import *


def read_ps_voltage(file):
    read_data = np.genfromtxt(file, skip_header=1, delimiter=',')
    return read_data[:, 1]

# Messungen

Wir haben angefangen 20 verschiedene Entfernungen zu messen und diese mit der Picoscope Software in 20 einzelne CSV Dateien gespeichert.

In [ ]:
files = glob.glob("cleaned/*cm*.csv")

# [reference in mm, all voltage values]
data = {
    100: read_ps_voltage(files[0]),
    130: read_ps_voltage(files[1]),
    160: read_ps_voltage(files[2]),
    180: read_ps_voltage(files[3]),
    220: read_ps_voltage(files[4]),
    245: read_ps_voltage(files[5]),
    250: read_ps_voltage(files[6]),
    280: read_ps_voltage(files[7]),
    300: read_ps_voltage(files[8]),
    340: read_ps_voltage(files[9]),
    380: read_ps_voltage(files[10]),
    410: read_ps_voltage(files[11]),
    460: read_ps_voltage(files[12]),
    490: read_ps_voltage(files[13]),
    520: read_ps_voltage(files[14]),
    580: read_ps_voltage(files[15]),
    620: read_ps_voltage(files[16]),
    650: read_ps_voltage(files[17]),
    690: read_ps_voltage(files[18]),
    710: read_ps_voltage(files[19]),
}

data_mean = {k: np.mean(v) for k, v in data.items()}

measurements = np.array(list(data_mean.items()))
distances = measurements[:, 0]
voltages = measurements[:, 1]

# Plotten und Logarithm

Wir können jetzt diese daten plotten. Sie haben die form $y = x^a$ deswegen müssen wir sie logarithmieren.

In [ ]:
subplot(1, 2, 1)
scatter(voltages, distances)

title("Original")
ylabel("Distance (mm)")
xlabel("Voltage")

log_distances = np.log(distances)
log_voltages = np.log(voltages)

subplot(1, 2, 2)
scatter(log_voltages, log_distances)

title("Logarithm")
ylabel("Distance (mm)")
xlabel("Voltage")

show()

# Lineare Regression

Jetzt können wir für die lineare Regression $a$ und $b$ berechnen.

Wir verwenden:

$a=\frac{\sum_{i=1}^{n}(x_i-\bar{x})*(y_i-\bar{y})}{\sum_{i=1}^{n}(x_i-\bar{x})^2}$ und $b=\bar{y}-a*\bar{x}$

In [ ]:
x = log_voltages
y = log_distances

x_mean = np.mean(x)
y_mean = np.mean(y)

a = np.divide(np.sum((x - x_mean) * (y - y_mean)),
              np.sum((x - x_mean) ** 2))

b = y_mean - a * x_mean

Das plotten wir noch.

In [ ]:
lin = np.linspace(-1, 1)
y_lin = a * lin + b

figure()

scatter(log_voltages, log_distances)
plot(lin, y_lin, 'r')

# TODO labels drehen
title("Logarithm")
xlabel("Distance (mm)")
ylabel("Voltage")
show()

Wir rechnen die lineare regression in eine exponentielle regression um indem wir $y = e^b * x^a$ anwenden.

In [ ]:
def exp_regression(x):
    return np.exp(b) * x ** a

xc = np.linspace(0.4, 2)
yc = exp_regression(xc)

figure()

scatter(voltages, distances)
plot(xc, yc, 'r')

# TODO labels drehen
title("Measurements")
xlabel("Distance (mm)")
ylabel("Voltage")
show()


# Fehlerrechnung



In [ ]:
# wahrer wert
dina4_breite_mm = 210

# gemessene werte
measured_dina4_breite_v = { "data": read_ps_voltage("./cleaned/dina4_breite_cleaned.csv") }

measured_dina4_breite_v["mean"] = np.mean(measured_dina4_breite_v["data"])
measured_dina4_breite_v["std"] = np.std(measured_dina4_breite_v["data"])
measured_dina4_breite_v["stdmean"] = measured_dina4_breite_v["std"] / np.sqrt(measured_dina4_breite_v["data"].size)

## 68 % Vertrauensbereich

In [ ]:
print("+-", str(measured_dina4_breite_v["stdmean"]))

## 95 % Vertrauensbereich

In [ ]:
print("+-", str(2*measured_dina4_breite_v["stdmean"]))

# Flächenmessung

Ein DIN A4 Blatt hat das Format $297 mm * 210 mm$

Die fläche sollte $62370mm^2$.

In [153]:
estimated_width = exp_regression(measured_dina4_breite_v["mean"])
estimated_height = estimated_width * 1.4142
flaeche = estimated_width * estimated_height
flaeche

60009.60425731316